In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
# Class count
count_class_0, count_class_1 = train.target.value_counts()
print(count_class_0); print(count_class_1)

# Divide by class
df_class_0 = train[train['target'] == 0]
df_class_1 = train[train['target'] == 1]

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
train = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(train.target.value_counts())

#train.target.value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
print(train.head())
print(type(train))
print(train.shape)
print(test.head())
print(type(test))
print(test.shape)

In [ ]:
X_train, y_train = train['question_text'].values, train['target'].values
X_test = test['question_text'].values
X_test_qid = test['qid'].values

In [ ]:
import re

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()   # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(" ", text)     # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub("", text)     # delete symbols which are in BAD_SYMBOLS_RE from text
    
    
    resultwords = [word for word in text.split() if word not in STOPWORDS]  # delete stopwords from text
    text = ' '.join(resultwords)
    
    return text

In [ ]:
X_train = [text_prepare(x) for x in X_train]
X_test = [text_prepare(x) for x in X_test]

In [ ]:
print(X_train[1])
print(X_test[1])

In [ ]:
words_counts = {}


from collections import Counter
counts = Counter()
for sentence in X_train:
  counts.update(word for word in sentence.split())

words_counts = counts

print(words_counts)
print(len(words_counts.keys()))


In [ ]:
DICT_SIZE = 100
WORDS_TO_INDEX = dict(sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:DICT_SIZE]) ####### YOUR CODE HERE #######
i=0
for key, value in WORDS_TO_INDEX.items():
    WORDS_TO_INDEX[key] = i
    i = i+1

ALL_WORDS = WORDS_TO_INDEX.keys()
print(WORDS_TO_INDEX)

def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    
    result_vector = np.zeros(dict_size)
   
    for word in text.split():      
      if word in words_to_index.keys():        
        result_vector[words_to_index.get(word)] += 1
    #result_vector = np.array(result_vector)
    print(result_vector)
    return result_vector

In [ ]:
from scipy import sparse as sp_sparse

In [ ]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
#X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
#print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

In [ ]:
print('X_train shape ', X_train_mybag.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

def train_classifier(X_train, y_train):
    """
      X_train, y_train — training data
      
      return: trained classifier
    """
    
    
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    return logreg


In [ ]:
classifier_mybag = train_classifier(X_train_mybag, y_train)

print(type(classifier_mybag))

In [ ]:
y_test_predicted_labels_mybag = classifier_mybag.predict(X_test_mybag)
y_test_predicted_scores_mybag = classifier_mybag.decision_function(X_test_mybag)

In [ ]:
final_output = {"qid": "prediction"}
i=0
for qid in X_test_qid:
    final_output[qid] = y_test_predicted_labels_mybag[i]
    i = i+1

In [ ]:
import csv
with open('submission.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in final_output.items():
       writer.writerow([key, value])